# 1. Importing modules and functions

In [89]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from molvs import standardize_smiles
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
import shap
from tqdm.notebook import tqdm

# 2.Data entry and curation work set

In [90]:
uploaded_file_ws="datasets/HDAC6_work.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  3083 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [91]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [92]:
len(y_tr)

3083

# 3.Standardization SDF file for work set

In [93]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  3083 molecules


In [94]:
records_ws = []
for i in range(len(moldf_ws)):
    record = Chem.MolToSmiles(moldf_ws[i])
    records_ws.append(record)

In [95]:
df = pd.DataFrame(records_ws, columns=["Smiles"])
df.to_csv('datasets/molecule_ws.smi', sep=',', index=False, header=False)

# 4.Data entry and curation test set

In [96]:
uploaded_file_ts="datasets/HDAC6_test.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  771 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [97]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [98]:
len(y_ts)

771

# 5.Standardization SDF file for test set

In [99]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  771 molecules


In [100]:
records_ts = []
for i in range(len(moldf_ts)):
    record = Chem.MolToSmiles(moldf_ts[i])
    records_ts.append(record)

In [101]:
df_ts = pd.DataFrame(records_ts, columns=["Smiles"])
df_ts.to_csv('datasets/molecule_ts.smi', sep=',', index=False, header=False)

# 6.Descriptor calculation for work set

In [17]:
fp

{'AtomPairs2DCount': 'fingerprints_xml\\AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': 'fingerprints_xml\\AtomPairs2DFingerprinter.xml',
 'EState': 'fingerprints_xml\\EStateFingerprinter.xml',
 'CDKextended': 'fingerprints_xml\\ExtendedFingerprinter.xml',
 'CDK': 'fingerprints_xml\\Fingerprinter.xml',
 'CDKgraphonly': 'fingerprints_xml\\GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': 'fingerprints_xml\\KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': 'fingerprints_xml\\KlekotaRothFingerprinter.xml',
 'MACCS': 'fingerprints_xml\\MACCSFingerprinter.xml',
 'PubChem': 'fingerprints_xml\\PubchemFingerprinter.xml',
 'SubstructureCount': 'fingerprints_xml\\SubstructureFingerprintCount.xml',
 'Substructure': 'fingerprints_xml\\SubstructureFingerprinter.xml'}

## KlekotaRoth FPs

In [163]:
from padelpy import padeldescriptor

fingerprint = 'KlekotaRoth'

fingerprint_output_file = ''.join([fingerprint,'.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='datasets/molecule_ws.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

In [102]:
descriptors_KlekotaRoth = pd.read_csv('KlekotaRoth.csv')

In [103]:
descriptors_KlekotaRoth

,Name,KRFP1,KRFP2,KRFP3,KRFP4,KRFP5,KRFP6,KRFP7,KRFP8,KRFP9,...,KRFP4851,KRFP4852,KRFP4853,KRFP4854,KRFP4855,KRFP4856,KRFP4857,KRFP4858,KRFP4859,KRFP4860
0,AUTOGEN_molecule_ws_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AUTOGEN_molecule_ws_2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AUTOGEN_molecule_ws_3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AUTOGEN_molecule_ws_4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AUTOGEN_molecule_ws_5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,AUTOGEN_molecule_ws_3079,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3079,AUTOGEN_molecule_ws_3080,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3080,AUTOGEN_molecule_ws_3081,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3081,AUTOGEN_molecule_ws_3082,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
x_tr = descriptors_KlekotaRoth.drop('Name', axis=1)

In [105]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

In [35]:
savetxt('Models/Padels/x_tr_KlekotaRoth.csv', y_tr, delimiter=',')

In [106]:
x_tr.shape

(3083, 4860)

# 7.Descriptor calculation for test set

In [179]:
from padelpy import padeldescriptor

fingerprint = 'KlekotaRoth'

fingerprint_output_file = ''.join([fingerprint,'_ts.csv']) #KlekotaRoth.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='datasets/molecule_ts.smi', 
                d_file=fingerprint_output_file, #'KlekotaRoth.csv'
                #descriptortypes='KlekotaRoth.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=False,
                fingerprints=True)

In [107]:
descriptors_KlekotaRoth_ts = pd.read_csv('KlekotaRoth_ts.csv')

In [108]:
x_ts = descriptors_KlekotaRoth_ts.drop('Name', axis=1)

In [109]:
x_ts.shape

(771, 4860)

# load the models from disk

In [110]:
best_svm = pickle.load(open('Models/Padels/HDAC6_SVM_KlekotaRoth.pkl', 'rb'))

In [111]:
best_gbr = pickle.load(open('Models/Padels/HDAC6_GBR_KlekotaRoth.pkl', 'rb'))

In [112]:
best_MLPR = pickle.load(open('Models/Padels/HDAC6_MLPR_KlekotaRoth.pkl', 'rb'))

# Prediction for CV

In [113]:
y_pred_ws_GBR = best_gbr.predict(x_tr)

In [114]:
y_pred_ws_svm = best_svm.predict(x_tr)

In [115]:
y_pred_con_ws=(y_pred_ws_GBR+y_pred_ws_svm)/2

In [116]:
R2_WS = round(r2_score(y_tr, y_pred_con_ws), 2)
R2_WS

0.93

In [117]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con_ws)), 2)
RMSE_WS

0.46

In [118]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [119]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [120]:
y_pred_CV_gbr = cross_val_predict(best_gbr, x_tr, y_tr, cv=cv)

In [121]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, x_tr, y_tr, cv=cv)

# For all models

In [122]:
y_pred_con=(y_pred_CV_svm+y_pred_CV_gbr+y_pred_CV_MLPR)/3

In [123]:
Q2_CV = round(r2_score(y_tr, y_pred_con), 2)
Q2_CV

0.66

In [124]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con)),2)
RMSE_CV

0.69

# two models: svm+gbr

In [125]:
y_pred_con_without_MLPR=(y_pred_CV_svm+y_pred_CV_gbr)/2

In [126]:
Q2_CV = round(r2_score(y_tr, y_pred_con_without_MLPR), 2)
Q2_CV

0.67

In [127]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con_without_MLPR)),2)
RMSE_CV

0.68

# Prediction for test set's molecules

In [128]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [129]:
y_pred_svm = best_svm.predict(x_ts)

In [130]:
y_pred_gbr = best_gbr.predict(x_ts)

In [131]:
y_pred_MLPR = best_MLPR.predict(x_ts)

In [133]:
y_pred_con_ts=(y_pred_svm+y_pred_gbr)/2

In [134]:
Q2_TS = round(r2_score(y_ts, y_pred_con_ts), 2)
Q2_TS

0.71

In [135]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_con_ts)), 2)
RMSE_TS

0.66

# Estimating applicability domain. Method - Euclidian distances, K=1

In [136]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [137]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,3073,3074,3075,3076,3077,3078,3079,3080,3081,3082
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.872983,5.099020,3.872983,2.236068,5.385165,2.000000,3.605551,4.358899,3.162278,6.082763,...,2.828427,3.000000,2.645751,2.236068,2.236068,3.000000,2.645751,4.898980,1.732051,5.000000
2,6.000000,5.385165,4.582576,2.645751,6.082763,5.656854,5.291502,5.567764,6.782330,6.244998,...,3.872983,4.242640,3.316625,2.645751,2.236068,4.898980,2.828427,4.898980,3.000000,5.099020
3,6.164414,5.385165,4.690416,2.645751,6.403124,5.916080,5.385165,5.567764,7.141428,6.324555,...,4.795832,4.582576,3.872983,2.828427,2.236068,5.000000,3.162278,5.000000,3.000000,5.099020
4,6.324555,5.385165,4.795832,2.645751,6.403124,6.000000,5.477226,5.656854,7.211102,6.324555,...,4.795832,4.898980,4.472136,3.000000,2.645751,5.000000,3.316625,5.000000,3.162278,5.099020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,13.190906,12.124355,13.453624,13.341664,13.152946,12.727922,12.649111,13.304134,13.379088,12.845233,...,12.767145,12.961481,12.569805,13.000000,13.266500,13.076696,12.449900,13.674794,12.409674,13.266500
3079,13.190906,12.206555,13.453624,13.341664,13.152946,12.727922,12.649111,13.304134,13.453624,12.845233,...,12.806249,12.961481,12.609520,13.000000,13.266500,13.076696,12.489996,13.674794,12.449900,13.266500
3080,13.190906,12.206555,13.453624,13.341664,13.228757,12.727922,12.649111,13.304134,13.490738,12.845233,...,12.845233,12.961481,12.961481,13.000000,13.341664,13.076696,12.649111,13.674794,12.449900,13.266500
3081,13.190906,12.409674,13.490738,13.341664,13.453624,12.767145,12.727922,13.416408,13.564660,12.845233,...,12.884099,12.961481,13.038404,13.114877,13.379088,13.076696,12.727922,13.711309,12.489996,13.304134


In [138]:
similarity= neighbors_k

In [139]:
Dmean=np.mean(similarity[1,:])

In [140]:
round(Dmean, 2)

2.73

In [141]:
std=np.std(similarity[1,:])

In [142]:
round(std, 2)

1.24

In [143]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

3.34


In [144]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [145]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,761,762,763,764,765,766,767,768,769,770
0,2.449490,3.000000,3.605551,6.557438,5.196152,1.000000,2.645751,2.645751,2.645751,3.464102,...,2.449490,1.732051,3.464102,2.000000,1.732051,3.741657,2.828427,2.828427,3.162278,2.236068
1,2.828427,3.162278,4.000000,7.071068,5.291502,3.000000,3.000000,2.645751,2.645751,3.872983,...,2.645751,2.645751,4.000000,2.000000,1.732051,4.000000,4.123106,3.316625,3.605551,2.449490
2,3.000000,4.472136,4.582576,7.141428,5.477226,6.557438,3.000000,3.464102,3.316625,4.000000,...,3.000000,2.828427,4.242640,2.645751,1.732051,4.123106,4.242640,3.605551,3.605551,2.449490
3,3.605551,6.082763,4.898980,7.348469,5.477226,6.855655,3.162278,3.605551,4.472136,4.242640,...,3.000000,3.000000,4.358899,2.828427,2.000000,4.472136,4.242640,3.872983,3.741657,2.645751
4,4.242640,6.164414,5.000000,7.416198,5.567764,6.928203,3.316625,3.605551,4.690416,5.830952,...,3.162278,3.000000,4.472136,3.000000,2.236068,4.472136,4.242640,3.872983,4.123106,2.828427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,12.845233,12.000000,12.884099,12.529964,12.727922,13.190906,12.922848,12.124355,13.379088,13.000000,...,13.453624,11.916375,13.527749,12.727922,13.190906,11.832160,13.228757,13.266500,12.649111,12.409674
3079,12.884099,12.124355,12.922848,12.529964,12.767145,13.266500,13.000000,12.124355,13.379088,13.000000,...,13.490738,12.083046,13.564660,12.845233,13.190906,11.874342,13.228757,13.304134,12.688578,12.449900
3080,12.884099,12.206555,12.922848,12.609520,12.767145,13.379088,13.038404,12.165525,13.379088,13.000000,...,13.490738,12.083046,13.638182,12.845233,13.266500,11.958261,13.228757,13.304134,12.727922,12.489996
3081,12.884099,12.369317,12.961481,12.609520,12.767145,13.379088,13.038404,12.328828,13.416408,13.000000,...,13.490738,12.124355,13.674794,13.114877,13.304134,11.958261,13.228757,13.304134,12.727922,12.569805


In [146]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.449 3.    3.606 6.557 5.196 1.    2.646 2.646 2.646 3.464 3.317 2.449
 5.568 2.    2.646 2.646 2.449 5.099 3.606 3.317 4.123 2.828 5.477 2.
 6.245 4.    2.236 5.568 3.162 3.873 4.123 7.071 3.873 3.    2.236 1.732
 4.359 2.646 2.    4.583 5.    3.    3.464 4.69  3.606 2.828 3.464 2.449
 2.    1.    3.606 2.646 4.899 4.    3.742 1.732 2.646 2.646 5.657 3.
 2.828 2.646 3.606 2.646 3.742 4.123 3.162 2.646 1.732 2.646 2.449 1.
 2.449 4.899 2.    1.414 3.    2.646 6.164 3.    1.    3.317 5.    1.732
 4.69  2.646 1.414 1.414 4.583 1.414 3.    2.449 3.162 2.    2.828 6.403
 5.196 3.464 2.236 3.873 2.449 5.831 2.    2.646 4.243 4.123 2.    2.828
 2.646 3.162 2.449 2.646 3.162 3.    3.873 3.    3.464 2.    2.236 2.449
 3.    1.    3.162 3.162 3.317 2.646 3.    2.828 3.162 3.742 2.646 1.414
 2.449 1.732 6.403 2.449 2.449 3.606 2.    1.414 3.162 2.    3.317 0.
 4.243 2.236 1.732 1.732 3.742 1.    2.646 0.    2.449 4.359 4.359 2.236
 1.732 1.732 4.    1.732 3.464 2.449 3.162 0.    2.    1.732 3.

In [147]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True False False False  True  True  True  True False  True  True
 False  True  True  True  True False False  True False  True False  True
 False False  True False  True False False False False  True  True  True
 False  True  True False False  True False False False  True False  True
  True  True False  True False False False  True  True  True False  True
  True  True False  True False False  True  True  True  True  True  True
  True False  True  True  True  True False  True  True  True False  True
 False  True  True  True False  True  True  True  True  True  True False
 False False  True False  True False  True  True False False  True  True
  True  True  True  True  True  True False  True False  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True  True False  True  True False  True  True  True  True  True  True
 False  True  True  True False  True  True  True  True False False  True
  True  True False  True False  True  True  True  T

In [148]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.74189364461738


In [149]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   1   5   6   7   8  10  11  13  14  15  16  19  21  23  26  28  33
  34  35  37  38  41  45  47  48  49  51  55  56  57  59  60  61  63  66
  67  68  69  70  71  72  74  75  76  77  79  80  81  83  85  86  87  89
  90  91  92  93  94  98 100 102 103 106 107 108 109 110 111 112 113 115
 117 118 119 120 121 122 123 124 125 126 127 128 130 131 132 133 135 136
 138 139 140 141 142 143 145 146 147 149 150 151 152 155 156 157 159 161
 162 163 164 165 168 169 170 171 172 173 174 175 176 180 181 183 184 185
 187 189 190 191 192 193 195 196 197 198 199 200 202 203 204 205 206 207
 208 209 210 211 212 214 216 217 219 220 222 223 224 225 226 228 229 230
 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249
 251 252 253 254 255 256 257 259 261 262 263 264 265 266 268 270 271 273
 274 275 277 278 279 280 281 284 285 286 288 289 290 291 293 294 295 297
 298 299 300 302 305 306 308 309 310 311 312 313 314 315 316 317 318 320
 321 324 32

In [150]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [151]:
y_pred_con_ad=list(y_pred_con_ts)

In [152]:
y_pred_con_ad[:] = [x for i,x in enumerate(y_pred_con_ad) if i not in out_Ad]

In [153]:
len(y_pred_con_ad)

572

In [154]:
y_ts_ad=list(y_ts)

In [155]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [156]:
len(y_ts_ad)

572

In [157]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_con_ad), 2)
Q2_TS

0.74

In [158]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_con_ad)), 2)
RMSE_TS

0.64